## Veiew DB in panadas

In [1]:
import sqlite3
import pandas as pd

dat = sqlite3.connect('option_strategies.db')
query = dat.execute("SELECT * From option_strategies")
cols = [column[0] for column in query.description]
results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
results

,id,scrape_date,strategy_type,tab_name,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,item_id,options_expiry_date,date_info,timestamp_of_trigger,strategy_status,price_when_triggered,premium_received
0,1,2025-03-10T08:41:01.483574,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,GOOGL,182.37,sell 192.5 - buy 202.5,202.5,192.5,76.0,1,2025-03-21,March,None,None,NaN,NaN
1,2,2025-03-10T08:41:01.574518,Bear Call,Mild Risk 95-97% accuracy > shorter expiry,MSFT,404.89,sell 425.0 - buy 435.0,435.0,425.0,124.0,2,2025-03-21,March,None,None,NaN,NaN
2,3,2025-03-10T08:41:04.952613,Bear Call,Minimal Risk 97-99% accuracy > shorter expiry,UNH,507.59,sell 535.0 - buy 545.0,545.0,535.0,93.0,1,2025-03-21,March,None,None,NaN,NaN
3,4,2025-03-10T08:41:08.288082,Bear Call,Mild Risk 95-97% accuracy > longer expiry,UNH,504.95,sell 540.0 - buy 550.0,550.0,540.0,79.0,1,2025-03-28,March,None,None,NaN,NaN
4,5,2025-03-10T08:41:11.642884,Bear Call,Minimal Risk 97-99% accuracy > longer expiry,UNH,507.27,sell 545.0 - buy 555.0,555.0,545.0,70.0,1,2025-03-28,March,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,2025-03-17T06:00:29.608166,Bull Put,Minimal Risk 97-99% accuracy > shorter expiry,META,587.53,sell 540.0 - buy 530.0,530.0,540.0,118.0,1,2025-03-28,March,None,None,NaN,NaN
78,79,2025-03-17T06:00:33.145132,Bull Put,Mild Risk 95-97% accuracy > longer expiry,JNJ,158.83,sell 152.5 - buy 143.0,143.0,152.5,65.0,1,2025-04-04,March,None,None,NaN,NaN
79,80,2025-03-17T06:00:33.284044,Bull Put,Mild Risk 95-97% accuracy > longer expiry,V,325.48,sell 315.0 - buy 305.0,305.0,315.0,164.0,2,2025-04-04,March,None,None,NaN,NaN
80,81,2025-03-17T06:00:36.863225,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,META,580.89,sell 540.0 - buy 530.0,530.0,540.0,187.0,1,2025-04-04,March,None,None,NaN,NaN


In [18]:
results[~results['timestamp_of_trigger'].isnull()]

,id,scrape_date,strategy_type,tab_name,ticker,trigger_price,strike_price,strike_buy,strike_sell,estimated_premium,item_id,options_expiry_date,date_info,timestamp_of_trigger,strategy_status,price_when_triggered,premium_received
8,9,2025-03-10T08:41:21.213768,Bull Put,Mild Risk 95-97% accuracy > shorter expiry,WMT,89.68,sell 86.0 - buy 75.0,75.0,86.0,58.0,4,2025-03-21,March,2025-03-10T14:51:27.476449,premium too low,87.97,NaN
12,13,2025-03-10T08:41:28.351534,Bull Put,Mild Risk 95-97% accuracy > longer expiry,META,589.67,sell 540.0 - buy 530.0,530.0,540.0,97.0,2,2025-03-28,March,2025-03-10T14:57:11.292093,premium too low,589.12,NaN
14,15,2025-03-10T08:41:28.538762,Bull Put,Mild Risk 95-97% accuracy > longer expiry,WMT,89.29,sell 86.0 - buy 80.0,80.0,86.0,70.0,4,2025-03-28,March,2025-03-10T14:51:25.356915,premium too low,87.97,NaN
16,17,2025-03-10T08:41:32.218233,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,QQQ,490,sell 460.0 - buy 450.0,450.0,460.0,273.0,2,2025-03-28,March,2025-03-10T08:46:27.463750,premium too low,488.20,NaN
17,18,2025-03-10T08:41:32.354756,Bull Put,Minimal Risk 97-99% accuracy > longer expiry,TQQQ,62.25,sell 60.0 - buy 50.0,50.0,60.0,210.0,3,2025-03-28,March,2025-03-10T14:33:41.525971,premium too low,61.69,NaN
46,47,2025-03-12T06:02:01.555882,Bull Put,Minimal Risk 97-99% accuracy > shorter expiry,JNJ,163.62,sell 157.5 - buy 148.0,148.0,157.5,72.0,1,2025-03-28,March,2025-03-12T21:19:42.678459,order placed,162.70,60.0


## Delete ALL rows in SQLite database

In [32]:

import sqlite3

conn = sqlite3.connect('option_strategies.db')
c = conn.cursor()

# Delete all rows from table
c.execute('DELETE FROM option_strategies;',)

print('We have deleted', c.rowcount, 'records from the table.')

# Commit the changes to db			
conn.commit()
# Close the connection
conn.close()

We have deleted 18 records from the table.


## Delete table

In [33]:

# importing sqlite module 
import sqlite3 
  
# create connection to the  
# database geek 
connection = sqlite3.connect('option_strategies.db') 
  
# drop table 
connection.execute("DROP TABLE option_strategies") 
  
print("data dropped successfully") 
  
# close the connection 
connection.close() 

data dropped successfully


## Update Row

In [37]:
import sqlite3

sql  = 'UPDATE option_strategies SET trigger_price=? WHERE id = ?'

try:
    with sqlite3.connect('option_strategies.db') as conn:
        cur = conn.cursor()
        cur.execute(sql, (490, 17))
        conn.commit()
except sqlite3.OperationalError as e:
    print(e)